In [3]:
from github import Github
import pandas as pd 
import itertools
import json
import requests

In [4]:
%store -r access_token
g = Github(access_token)

In [5]:
g.rate_limiting

(0, 5000)

##### This below code is to get the repo count , number of pages , final_repo_list , list_with_lesson & list_without_lesson.

In [ ]:
 org=g.get_organization("Carpentries-Incubator")

In [ ]:
repo = g.get_organization("carpentries-incubator").get_repos()
repo.totalCount

In [ ]:
page_num = 0
for i in range(0,10):
    if len(repo.get_page(i))==0:
        print(i, "======================Empty")
        page_num = i
        break
    else:
        pass

In [ ]:
repo_list = list()
for i in range(0,page_num):
    repo_list.append(repo.get_page(i))

In [ ]:
final_repo_list = list((itertools.chain.from_iterable(repo_list)))

In [ ]:
list_without_lesson=list()
list_with_lesson=list()
for repo in final_repo_list:
    tags=repo.get_topics()
    if "lesson" in tags:
        list_with_lesson.append(repo)
    else:
        list_without_lesson.append(repo)
print("List of Repository with lesson:")   
for repo in list_with_lesson:
      print(repo)
print("\nList of Repository without lesson\n")
for repo in list_without_lesson:
      print(repo)

In [ ]:
# This code has been added here to figure out a way to remove null value from issue/pr/commit users information.So this could be ignored 

for r in list_with_lesson:
    if r.name=='deep-learning-intro':
        issue=r.get_issues(state='open')
        for i in issue:
            print(i.title,i.user.login)
        

#### This approach is followed to retrieve data of each repo in seperate files that would encompass commits, PRs, issues, stages & labels. 

In [1]:
import requests
repo=list_with_lesson[83]
HEADERS = {'Authorization': 'token ' + access_token}

NameError: name 'list_with_lesson' is not defined

In [ ]:
counter=0
commits_dict={}

In [ ]:
issue_list=list()
label_list=list()
pr_list=list()
# releases=list()
commits = repo.get_commits()
pr=repo.get_pulls(state='open')
issues=repo.get_issues(state='open')
topics=repo.get_topics()
labels=repo.get_labels()
releases=repo.get_releases()
owner=repo.owner.login
def commit_user_details(c):
    # first we try to retrieve details via PyGithub. This costs less API calls.
    try: 
        author = c.author.name
        committer = c.committer.name
        # The above case will raise an exception if c.author or c.committer is None.
        # This will be the case when the user does not have a GitHub account.
        # There may be another special case, when a user has a GitHub user name, but
        # does not specify their name. We need to detect it as well.
        if author == None or committer == None:
            raise Exception()
        return [author,committer]
    except:
        # If at least one of the names is not known, use GitHub API via `requests` library.
        # This will be slower and will cost some API calls, but at least we are spending
        # one API call per each commit instead of two.
        # Based on solution for https://github.com/jbon/Github-Social-Network-Analysis/issues/15
        commit_json = requests.get(c.url,headers=HEADERS).json()["commit"]
        author = commit_json['author']['name']
        committer = commit_json['committer']['name']
#         print(author,committer)
        return [author,committer]


# To retrieve PRs and usernames .

for p in pr:
#     print(p.id)
    pr_list.append({'id':p.id,'pr_name':p.user.login})
    
    
counter=0
commits_dict=dict()
for c in commits:


#     print(get_user_details(c))
    commits_dict[c.sha] = commit_user_details(c)
    
    
# print("retrieved", counter, "commits") 

# To retrieve issues and user names

for issue in issues:
    issue_list.append({'title': issue.title, 'user_name':issue.user.login})
    
# To retrieve life cycle stages of the lessons.
stage=None
for stage in topics:
    if stage in ["pre-alpha", "alpha", "beta", "stable"]:
        break;
        

# To retrieve Labels of the lessons 

for label in labels:
     label_list.append(label.name)
 
counter=0



    


In [2]:
with open(repo.name+'.json', 'w', encoding='utf-8') as f: 
        json.dump({'commits_dict':commits_dict,'pr_dict':pr_list, 'issue_dict':issue_list,'stage':stage,'owner':owner,'label':label_list}, f, ensure_ascii=False, indent=4) 

NameError: name 'repo' is not defined

In [ ]:
with open(repo.name+'.json', "r") as read_file: 
         commits_dict_1 = json.load(read_file) 